In [1]:
%matplotlib Qt
# %matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from sklearn.model_selection import train_test_split

from modules import Perceptron

In [3]:
data = pd.read_csv('data/BostonHousing.csv')
data.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [4]:
X = data[['rm', 'age']].values
Y = data['medv'].values


In [5]:
# Normalize the data ==> Xnorm = (X−mean) / std
X_mean = np.mean(X, axis=0)
X_std = np.std(X, axis=0)
X = (X - X_mean) / X_std

Y_mean = np.mean(Y)
Y_std = np.std(Y)
Y = (Y - Y_mean) / Y_std

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True)

In [7]:
learning_rate_w = 0.0001
learning_rate_b = 0.001
Epoch = 0
stop_condition = False
loss_threshold = 0.001  # Threshold for stopping condition

In [8]:
# Create and train the model
perceptron = Perceptron(input_size=2, lr_w=learning_rate_w, lr_b=learning_rate_b, epoch=Epoch, loss_threshold=loss_threshold)
perceptron.fit(X_train, Y_train)

Epoch 1, Loss: 1.1223723851551415
Epoch 2, Loss: 1.0710408109921938
Epoch 3, Loss: 1.028026631184246
Epoch 4, Loss: 0.990556186540791
Epoch 5, Loss: 0.9555424309733549
Epoch 6, Loss: 0.923010695059445
Epoch 7, Loss: 0.8923002213687229
Epoch 8, Loss: 0.8631133433906109
Epoch 9, Loss: 0.8355909187810694
Epoch 10, Loss: 0.8094720343715787
Epoch 11, Loss: 0.7849884544388702
Epoch 12, Loss: 0.7617795795575132
Epoch 13, Loss: 0.7397568438197932
Epoch 14, Loss: 0.7190467249552303
Epoch 15, Loss: 0.6995456141280354
Epoch 16, Loss: 0.6813410271190055
Epoch 17, Loss: 0.6643338806642638
Epoch 18, Loss: 0.6482459665088912
Epoch 19, Loss: 0.6329361414938632
Epoch 20, Loss: 0.618426144909066
Epoch 21, Loss: 0.6048857356931704
Epoch 22, Loss: 0.5921916575378904
Epoch 23, Loss: 0.5803622578834905
Epoch 24, Loss: 0.5693535624398519
Epoch 25, Loss: 0.5590461524343187
Epoch 26, Loss: 0.549295616203351
Epoch 27, Loss: 0.5401271298879322
Epoch 28, Loss: 0.5314842118872013
Epoch 29, Loss: 0.5233766729935557

In [9]:
# Evaluate
loss = perceptron.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")

Test Loss: 1.043764818900612


In [10]:
# Predict
Y_pred = perceptron.predict(X_train)


In [11]:
# Convert normalized data back to original scale
X_train_original = X_train * X_std + X_mean
Y_train_original = Y_train * Y_std + Y_mean
Y_pred_original = Y_pred * Y_std + Y_mean

In [12]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Scatter plot
ax.scatter(X_train_original[:, 0], X_train_original[:, 1], Y_train_original, color='blue', label='Training Data')

# Animation function
def update(num):
    ax.clear()
    ax.scatter(X_train_original[:, 0], X_train_original[:, 1], Y_train_original, color='blue', label='Training Data')
    
    # Update prediction for each frame
    Y_pred = np.dot(X_train, perceptron.w) + perceptron.b
    # Y_pred_original = Y_pred * Y_std + Y_mean

    # Create a meshgrid for the plane
    x_surf, y_surf = np.meshgrid(np.linspace(X_train_original[:, 0].min(), X_train_original[:, 0].max(), 100),
                                 np.linspace(X_train_original[:, 1].min(), X_train_original[:, 1].max(), 100))
    z_surf = perceptron.w[0] * (x_surf - X_mean[0]) / X_std[0] + perceptron.w[1] * (y_surf - X_mean[1]) / X_std[1] + perceptron.b
    z_surf = z_surf * Y_std + Y_mean
    
    ax.plot_surface(x_surf, y_surf, z_surf, color='red', alpha=0.5)
    ax.set_xlabel('ROOM')
    ax.set_ylabel('AGE')
    ax.set_zlabel('Average House Price')
    ax.set_title(f'Perceptron Linear Regression (Epoch: {num})\nBostonHousing')

ani = animation.FuncAnimation(fig, update, frames=len(perceptron.losses), repeat=False)

plt.show()